In [1]:
import requests as req #for request
from bs4 import BeautifulSoup #for html scraping
import pandas as pd #for dataframe

# Scraping Harga Emas

In [2]:
# Ekstrak Data
web = req.get('https://pusatdata.kontan.co.id/market/chart_logam_mulia/?startdate=2015-01-01&enddate=2022-03-20&logam=gold')

In [3]:
soup = BeautifulSoup(web.content, 'html.parser')
soup

 <div class="sec-pane">
<div class="clear-all high-25"></div>
<div class="myDivo" id="congenab">
<canvas id="line-chart" style="width:100%; height:420px;"></canvas>
<script>
          var tanggal = new Array();
                        tanggal.push('2015-01-02');
                        tanggal.push('2015-01-05');
                        tanggal.push('2015-01-06');
                        tanggal.push('2015-01-07');
                        tanggal.push('2015-01-08');
                        tanggal.push('2015-01-09');
                        tanggal.push('2015-01-12');
                        tanggal.push('2015-01-13');
                        tanggal.push('2015-01-14');
                        tanggal.push('2015-01-15');
                        tanggal.push('2015-01-16');
                        tanggal.push('2015-01-19');
                        tanggal.push('2015-01-20');
                        tanggal.push('2015-01-21');
                        tanggal.push('2015-01-22');
         

In [4]:
# Transaform Data
tanggal = []
keterangan = []
harga = []
body = soup.find_all(class_='tabel-body')
for x in body:
    tanggal.append(x.find(class_='kol-konten3-1').get_text())
    keterangan.append(x.find(class_='kol-konten3-2').get_text())
    harga.append(x.find(class_='kol-konten3-3').get_text())

In [5]:
data = {
    "Tanggal":tanggal,
    "Keterangan":keterangan,
    "Harga":harga
}
df = pd.DataFrame(data=data)

In [6]:
df.head()

,Tanggal,Keterangan,Harga
0,19/03/2022,BUYBACK,Rp 887.000
1,19/03/2022,GOLD,Rp 984.000
2,18/03/2022,BUYBACK,Rp 894.000
3,18/03/2022,GOLD,Rp 991.000
4,17/03/2022,BUYBACK,Rp 888.000


In [7]:
# Load Data
df.to_csv('hargaEmas.csv')

# Scraping Data Cuaca

In [8]:
import xmltodict
import json

In [9]:
# Ekstrak Data
cuaca = req.get('https://data.bmkg.go.id/DataMKG/MEWS/DigitalForecast/DigitalForecast-DIYogyakarta.xml')
dataCuaca = xmltodict.parse(cuaca.content,attr_prefix='')
dictCuaca = json.dumps(dataCuaca)
dictCuaca = json.loads(dictCuaca)

In [10]:
dt = dictCuaca['data']['forecast']

In [11]:
def kamus_cuaca():
    cuaca = {
        0 : "Cerah",
        1 : "Cerah Berawan",
        2 : "Cerah Berawan",
        3 : "Berawan",
        4 : "Berawan Tebal",
        100 : "Cerah",
        101 : "Cerah Berawan",
        102 : "Cerah Berawan",
        103 : "Berawan",
        104 : "Berawan Tebal",
        5 : "Udara Kabur",
        10 : "Asap",
        45 : "Kabut",
        60 : "Hujan Ringan",
        61 : "Hujan Sedang",
        63 : "Hujan Lebat",
        80 : "Hujan Lokal",
        95 : "Hujan Petir",
        97 : "Hujan Petir"
    }
    return cuaca

In [12]:
# Transfrom
dCuaca = kamus_cuaca()
_data = []
for i in dt['area']:
    x =  i['domain']
    weather = []
    area = {
            'source': 'BMKG (Badan Meteorologi, Klimatologi, dan Geofisika)',
            'id': i['id'],
            'latitude': i['latitude'],
            'longitude': i['longitude'],
            'coordinate': i['coordinate'],
            'kabupaten': i['description'],
            'propinsi': i['domain'],
            'waktu': dt['issue']
    }
    for x in i['parameter']:
        if x['id'] == 'weather':
            for y in x['timerange']:
                kd = y['value']['#text']
                ketCuaca = dCuaca[int(kd)]
                y['value']['#text'] = ketCuaca
                weather.append(y)
    temp = { 'weather':weather}    
    area.update(temp)
    _data.append(area)

In [13]:
_data

[{'source': 'BMKG (Badan Meteorologi, Klimatologi, dan Geofisika)',
  'id': '501186',
  'latitude': '-7.916666669',
  'longitude': '110.3167',
  'coordinate': '110.3167 -7.916666669',
  'kabupaten': 'Bantul',
  'propinsi': 'DI Yogyakarta',
  'waktu': {'timestamp': '20220320024244',
   'year': '2022',
   'month': '03',
   'day': '20',
   'hour': '02',
   'minute': '42',
   'second': '44'},
  'weather': [{'type': 'hourly',
    'h': '0',
    'datetime': '202203200000',
    'value': {'unit': 'icon', '#text': 'Berawan'}},
   {'type': 'hourly',
    'h': '6',
    'datetime': '202203200600',
    'value': {'unit': 'icon', '#text': 'Berawan'}},
   {'type': 'hourly',
    'h': '12',
    'datetime': '202203201200',
    'value': {'unit': 'icon', '#text': 'Hujan Sedang'}},
   {'type': 'hourly',
    'h': '18',
    'datetime': '202203201800',
    'value': {'unit': 'icon', '#text': 'Berawan'}},
   {'type': 'hourly',
    'h': '24',
    'datetime': '202203210000',
    'value': {'unit': 'icon', '#text': 'B

In [48]:
kabupaten = []
waktu = []
cuaca = []
jam = []
for d in _data:
    print(d['kabupaten'])
    for w in d['weather']:
        kabupaten.append(d['kabupaten'])
        waktu.append(d['waktu']['year']+'/'+d['waktu']['month']+'/'+d['waktu']['day']+' '+d['waktu']['hour']+':'+d['waktu']['minute'])
        print('Hours: '+w['h']+' - '+w['value']['#text'])
        jam.append(w['h'])
        cuaca.append(w['value']['#text'])
    print()

Bantul
Hours: 0 - Berawan
Hours: 6 - Berawan
Hours: 12 - Hujan Sedang
Hours: 18 - Berawan
Hours: 24 - Berawan
Hours: 30 - Berawan
Hours: 36 - Hujan Lebat
Hours: 42 - Berawan
Hours: 48 - Cerah Berawan
Hours: 54 - Berawan
Hours: 60 - Hujan Ringan
Hours: 66 - Cerah Berawan

Sleman
Hours: 0 - Berawan
Hours: 6 - Hujan Petir
Hours: 12 - Hujan Ringan
Hours: 18 - Berawan
Hours: 24 - Berawan
Hours: 30 - Berawan
Hours: 36 - Hujan Ringan
Hours: 42 - Berawan
Hours: 48 - Cerah Berawan
Hours: 54 - Berawan
Hours: 60 - Cerah Berawan
Hours: 66 - Cerah Berawan

Wates
Hours: 0 - Berawan
Hours: 6 - Berawan
Hours: 12 - Hujan Sedang
Hours: 18 - Berawan
Hours: 24 - Berawan
Hours: 30 - Berawan
Hours: 36 - Hujan Sedang
Hours: 42 - Berawan
Hours: 48 - Cerah Berawan
Hours: 54 - Berawan
Hours: 60 - Hujan Ringan
Hours: 66 - Cerah Berawan

Gunung Kidul
Hours: 0 - Berawan
Hours: 6 - Hujan Ringan
Hours: 12 - Hujan Lebat
Hours: 18 - Berawan
Hours: 24 - Berawan
Hours: 30 - Hujan Sedang
Hours: 36 - Hujan Petir
Hours: 42

In [52]:
dataCuaca = {
    'Kabupaten':kabupaten,
    'Tanggal Update':waktu,
    'Jam':jam,
    'Cuaca':cuaca
}

dfCuaca = pd.DataFrame(data=dataCuaca)
dfCuaca

,Kabupaten,Tanggal Update,Jam,Cuaca
0,Bantul,2022/03/20 02:42,0,Berawan
1,Bantul,2022/03/20 02:42,6,Berawan
2,Bantul,2022/03/20 02:42,12,Hujan Sedang
3,Bantul,2022/03/20 02:42,18,Berawan
4,Bantul,2022/03/20 02:42,24,Berawan
5,Bantul,2022/03/20 02:42,30,Berawan
6,Bantul,2022/03/20 02:42,36,Hujan Lebat
7,Bantul,2022/03/20 02:42,42,Berawan
8,Bantul,2022/03/20 02:42,48,Cerah Berawan
9,Bantul,2022/03/20 02:42,54,Berawan


In [53]:
dfCuaca[dfCuaca['Jam']=='18']

,Kabupaten,Tanggal Update,Jam,Cuaca
3,Bantul,2022/03/20 02:42,18,Berawan
15,Sleman,2022/03/20 02:42,18,Berawan
27,Wates,2022/03/20 02:42,18,Berawan
39,Gunung Kidul,2022/03/20 02:42,18,Berawan
51,Yogyakarta,2022/03/20 02:42,18,Berawan


In [27]:
# Load Data
dfCuaca.to_csv('cuacaDIY.csv')